Modelling a  handwritten digit classifier using Multi-layer Neural Network. The model will be trained to classify the images of handwritten digits into 10 classes( digits 0 -9).
We will be using [MNIST dataset of handwritten digits](http://yann.lecun.com/exdb/mnist/) for training the classifier.The dataset is a good example of real-world data and is widely used by Machine Learning community for learning techniques and pattern recognition methods.

MNIST dataset contains grayscale samples of handwritten digits of size 28 $\times$ 28. It is split into training set of 60,000 examples, and a test set of 10,000 examples. For this assignment, we will using a smaller subset of 1500 training samples, 500 validation samples and 1000 test samples.

In [ ]:
#initialization
from datasets import mnist
from matplotlib import pyplot as plt
import numpy as np
import numpy.testing as npt
import pytest
import random
import numpy.matlib 

random.seed(1)
np.random.seed(1)

train_samples = 1500
val_samples = 500
test_samples = 1000

digits = list(range(10))
trX, trY, tsX, tsY, valX, valY = mnist(train_samples,val_samples,test_samples, digits=digits)

 # Parameter Initialization

 Let us now define a function that can initialize the parameters of the Neural Network.
The network parameters are wrapped as dictionary elements that can easily be passed as function parameters while calculating gradients during back propogation.

1. The weight matrix is initialized with random values from a normal distribution of variance $1$. For example, to create a matrix $W$ of dimension $3 \times 4$, with values from a normal distribution with variance $1$,
we define $W = np.random.normal(size =(3,4))$.

2. Bias values are initialized with a vector of 0's.

The dimension of weight matrix for a layer $(l+1)$ is given by ( Number of neurons in $(l+1)$  X  Number of neurons in $l$ )

The dimension of bias vector for a layer $(l+1)$ is given by ( Number of neurons in $(l+1)$  X  Number of neurons in $1$ )

In [ ]:
def initialize(net_dims):
    numLayers = len(net_dims)
    parameters = {}
    for l in range(numLayers-1):    
        parameters["W"+str(l+1)] = np.random.normal(size=(net_dims[l+1], net_dims[l]))
        parameters["b"+str(l+1)] = np.zeros(net_dims[l+1]).reshape(net_dims[l+1], 1)
    return parameters

An Activation function takes an input from the previous layer and performs a certain fixed mathematical operation and the result is passed to the following layer.
1. ReLU or Rectified Linear Unit
ReLU (Rectified Linear Unit) is a piecewise linear function that will output the input if is positive, otherwise, it's output is zero.
\begin{equation*}
ReLU(x) = Max(0,x)
\end{equation*}
2. Linear activation
 Linear activation performs a simple linear operation of passing the input.
\begin{equation*}
Linear(x) = x\\
dx = 1
\end{equation*}

In [ ]:
def relu(Z):
    cache = {}
    features = Z.shape[0]
    samples = Z.shape[1]
    A = np.zeros((features, samples))
    for i in range(features) :
        for j in range(samples) :
            A[i][j] = np.maximum(0, Z[i][j])
    cache["Z"] = Z
    return A, cache      

def relu_der(dA, cache):
    dZ = np.array(dA, copy=True)
    Z = cache["Z"]
    features = dZ.shape[0]
    samples = dZ.shape[1]
    for i in range(features) :
        for j in range(samples) :
            dZ[i][j] = np.maximum(0, dZ[i][j])
    return dZ

In [ ]:
def linear(Z):
    A = Z
    cache = {}
    cache["Z"] = Z
    return A, cache

def linear_der(dA, cache):
    dZ = np.array(dA, copy=True)
    return dZ

# Loss function

The softmax activation is computed on the outputs from the last layer and the output label with the maximum probablity is predicted as class label. The softmax function can also be refered as  normalized exponential function which takes a vector of $n$ real numbers as input, and normalizes it into a probability distribution consisting of $n$ probabilities proportional to the exponentials of the input numbers. 

The input to the softmax function is the matrix of all the samples, $ Z = [ z^{(1)} , z^{(2)}, \ldots, z^{(m)} ] $, where $z^{(i)}$ is the $i^{th}$ sample of $n$ dimensions. We estimate the softmax for each of the samples $1$ to $m$. The softmax activation for $a^{(i)} = \text{softmax}(z^{(i)})$ is, 
\begin{equation}
a_k{(i)} = \frac{exp(z^{(i)}_k)}{\sum_{k = 1}^{n}exp(z^{(i)}_k)} \qquad \text{for} \quad 1\leq k\leq n
\end{equation}

The output of the softmax is $ A = [ a^{(1)} , a^{(2)} .... a^{(m)} ]$, where $a^{(i)} = [a^{(i)}_1,a^{(i)}_2, \ldots, a^{(i)}_n]^\top$.  In order to avoid floating point overflow, we subtract a constant from all the input components of $z^{(i)}$ before calculating the softmax. This constant is $z_{max}$, where, $z_{max} = max(z_1,z_2,...z_n)$. 
Note: 
\begin{equation}
a_k{(i)} = \frac{exp(z^{(i)}_k- z_{max})}{\sum_{k = 1}^{n}exp(z^{(i)}_k - z_{max})} \qquad \text{for} \quad 1\leq k\leq n
\end{equation}

If the output of softmax is given by $A$ and the ground truth is given by $Y = [ y^{(1)} , y^{(2)}, \ldots, y^{(m)}]$, the cross entropy loss between the predictions $A$ and groundtruth labels $Y$ is given by,

\begin{equation}
Loss(A,Y) = - \frac{1}{m} \sum_{i=1}^m \sum_{k=1}^{n}I \{ y^i = k \} \text{log}a_k^i
\end{equation}


where $I$ is the identity function given by 

\begin{equation}
I\{\text{condition}\} = 1, \quad \text{if condition = True}\\
I\{\text{condition}\} = 0, \quad \text{if condition = False}\\
\end{equation}

The derivative of the multiclass cross entropy loss can be given as the difference between the Activation output and ground truth. If $A$ is vector of $m$ samples , as $ A = [ a^{(1)} , a^{(2)} .... a^{(m)} ]$, the gradient of softmax is given by,

\begin{equation}
dZ =\frac{1}{m} (A -Y)
\end{equation}

In [ ]:
def softmax_cross_entropy_loss(Z, Y=np.array([])):
    n = Z.shape[0]
    m = Z.shape[1]
    A = np.zeros((n, m))
    zmax = np.max(Z, axis = 0)
    D = np.sum(np.exp(Z - zmax), axis=0)
    A = np.exp(Z - zmax)/D 
    loss = []
    if len(Y) > 0 : 
        loss = -1 * np.sum(np.multiply(np.sum(np.log(A)), np.where(A == Y, 1, 0)))/m   
    cache = {}
    cache["A"] = A
    return A, cache, loss

def softmax_cross_entropy_loss_der(Y, cache):
    A = cache["A"]
    m = Y.shape[1]
    dZ = (A - Y)/m  
    return dZ

In [ ]:
def one_hot(Y,num_classes):
    Y_one_hot = np.zeros((num_classes,Y.shape[1]))
    for i in range(Y.shape[1]):
        Y_one_hot[int(Y[0,i]),i] = 1
    return Y_one_hot

In [ ]:
def dropout_forward(A, drop_prob, mode='train'):        
        mask = np.random.rand(*A.shape) > drop_prob
        out = None       
        if mode == 'train':
            out = np.multiply(A, mask)
            out = out / (1-drop_prob)
        elif mode == 'test':
            out = A
        else:
            raise ValueError("Mode value not set, set it to 'train' or 'test'")
        cache = (mode, mask)
        return out, cache

def dropout_backward(cache, dout):
        dA = None
        mode, mask = cache
        if mode == 'train':
            dA = np.multiply(dout, mask)
        elif mode == 'test':
            dA = dout 
        else:
            raise ValueError("Mode value not set, set it to 'train' or 'test'")
        return dA

## Forward Propagation
One Layer
  
If the vectorized input to any layer of neural network is $A$ and the parameters of the layer is given by $(W,b)$ ,the output of the layer is,
\begin{equation}
Z = W A + b
\end{equation}

## Layer + Activation
In addition to layer, the following function also computes the activation of each layer which is given by,
\begin{equation}
Z = W X + b\\
A = \sigma (Z)
\end{equation}

depending on the activation choosen for the given layer, the $\sigma(.)$ can represent different operations.


In [ ]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = {}
    cache["A"] = A
    return Z, cache 

def layer_forward(A_prev, W, b, activation, drop_prob, mode):
    Z, lin_cache = linear_forward(A_prev, W, b)    
    if activation == "relu":
        A, act_cache = relu(Z)
        A, drop_cache =  dropout_forward(A, drop_prob, mode)        
    elif activation == "linear":
        A, act_cache = linear(Z)
        drop_cache = None     
    cache = {}
    cache["lin_cache"] = lin_cache
    cache["act_cache"] = act_cache
    cache["drop_cache"] = drop_cache    
    return A, cache

## Multi-Layers 

The forward layers are stacked to form a multi layer network. The number of layers used by the network can be inferred from the size of the $parameters$. If the number of items in the dictionary element $parameters$ is $2L$, then the number of layers will be $L$

During forward propagation , the input sample $A_0$ is fed into the first layer and the subsequent layers use the activation output from the previous layer as inputs.

Please note all the hidden layers use **ReLU** activation except the last layer which uses **Linear** activation.

In [ ]:
def multi_layer_forward(X, parameters,drop_prob, mode):
    L = len(parameters)//2  
    A = X
    caches = []
    
    for l in range(1,L):
        A, cache = layer_forward(A, parameters["W" + str(l)], parameters["b" + str(l)], "relu", drop_prob, mode)        
        caches.append(cache)
        
    AL, cache = layer_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "linear",drop_prob, mode)
    caches.append(cache)
    return AL, caches

# Backward Propagagtion (10 Points)

Performing back propagation through the layers and calculating the gradients of the network parameters $(dW,db)$.

If the derivative of the loss$\frac{dL}{dZ}$ is given as $dZ$ and network paramerters are given as $(W,b)$, the gradients $(dW,db)$ can be calculated as,

\begin{equation}
dA_{prev} = W^T dZ\\
dW = dZ A^T\\
db = \sum_{i=1}^{m} dz^{(i)}\\
\end{equation}

where $dZ = [dz^{(1)},dz^{(2)}, \ldots, dz^{(m)}]$ is the column vector of the gradient of loss in the kth layer.

#### Layer + Activation

In the below function, we also account for the activation while calculating the derivative.
We use the derivative functions defined earlier to calculate $(\frac{dL}{dZ})$ followed by back propagation.

In [ ]:
def linear_backward(dZ, cache, W, b):   
    A = cache['A']
    dA_prev = np.dot(W.T, dZ)
    dW = np.dot(dZ, A.T)
    n = dZ.shape[0]
    db = np.sum(dZ, axis=1).reshape(n, 1)   
    return dA_prev, dW, db

def layer_backward(dA, cache, W, b, activation):  
    lin_cache = cache["lin_cache"]
    act_cache = cache["act_cache"]
    drop_cache = cache["drop_cache"]
    if activation == "relu":        
        dA = dropout_backward(drop_cache, dA)
        dZ = relu_der(dA, act_cache)        
    elif activation == "linear":        
        dZ = linear_der(dA, act_cache)        
    dA_prev, dW, db = linear_backward(dZ, lin_cache, W, b)
    return dA_prev, dW, db

## Multi-layers

We have defined the required functions to handle back propagation for single layer. Now we will stack the layers together and perform back propagation on the entire network.

In [ ]:
def multi_layer_backward(dAL, caches, parameters):
    L = len(caches) 
    gradients = {}
    dA = dAL
    activation = "linear"
    for l in reversed(range(1,L+1)):
        dA, gradients["dW"+str(l)], gradients["db"+str(l)] = \
                    layer_backward(dA, caches[l-1], \
                    parameters["W"+str(l)],parameters["b"+str(l)],\
                    activation)
        activation = "relu"
    return gradients

## Prediction
Assembling the different parts of forward propagation into a single unit and use the ouput to make a prediction.

In [ ]:
def classify(X, parameters,mode,drop_prob):   
    A, caches = multi_layer_forward(X, parameters,drop_prob, mode)
    A_softmax, caches, loss = softmax_cross_entropy_loss(A)
    m = X.shape[1]
    YPred = np.argmax(A_softmax, axis=0).reshape((1,m))
    return YPred

## Momentum

A very popular technique that is used along with gradient descent is Momentum. Instead of using only the gradient of the current step to guide the search for minima, momentum also accumulates the gradient of the past steps to determine the direction of descent.

In [ ]:
def initialize_velocity(parameters):    
    L = len(parameters) // 2 
    v = {}
    
    for l in range(L):
        v["dW" + str(l + 1)] = np.zeros_like(parameters["W" + str(l+1)])
        v["db" + str(l + 1)] = np.zeros_like(parameters["b" + str(l+1)])
            
    return v

## Parameter updates

The parameter gradients $(dW,db)$ calculated during back propagation are used to update the values of the network parameters.

\begin{equation}
V_{t+1} = \beta  V_{t} +(1-\beta)\nabla J(\theta_t)\\
\theta_{t+1} =\theta_{t} -\alpha(V_{t+1}), \quad \theta \in \{ W,b \}
\end{equation}

Where $\alpha$ is the learning rate of the network and $\beta$ is the momentum parameter . As discussed in the lecture, decay rate is used to adjust the learning rate smoothly across the gradient curve to avoid overshooting.

In [ ]:
def update_parameters_with_momentum(parameters, gradients, epoch, v, beta, learning_rate, decay_rate=0.01):  
    alpha = learning_rate*(1/(1+decay_rate*epoch))
    L = len(parameters) // 2 # number of layers in the neural networks
    for i in range(L):
        v["dW"+str(i+1)] = beta * v["dW"+str(i+1)] + (1 - beta) * gradients["dW"+str(i+1)]
        v["db"+str(i+1)] = beta * v["db"+str(i+1)] + (1 - beta) * gradients["db"+str(i+1)]
        parameters["W"+str(i+1)] = parameters["W"+str(i+1)] - alpha * v["dW"+str(i+1)]
        parameters["b"+str(i+1)] = parameters["b"+str(i+1)] - alpha * v["db"+str(i+1)]
    return parameters, alpha, v

# Neural Network

Assembling all the components of the neural network together and define a complete training loop for the Multi-layer Neural Network.

In [ ]:
def multi_layer_network(X, Y,valX, valY, net_dims, drop_prob, mode, num_iterations=500, learning_rate=0.2, decay_rate=0.00005):

    parameters = initialize(net_dims)
    A0 = X
    costs = []
    val_costs = []
    num_classes = 10
    Y_one_hot = one_hot(Y,num_classes)
    valY_one_hot = one_hot(valY,num_classes)
    alpha = learning_rate
    beta = 0.9
    
    for ii in range(num_iterations):
        Z, cache_1 = multi_layer_forward(A0, parameters, drop_prob, mode)
        AL, cache_2, cost = softmax_cross_entropy_loss(Z, Y_one_hot) 
        dZ = softmax_cross_entropy_loss_der(Y_one_hot, cache_2)
        gradients = multi_layer_backward(dZ, cache_1, parameters)
        v = initialize_velocity(parameters)
        parameters, alpha, v = update_parameters_with_momentum(parameters, gradients, num_iterations, v, beta, learning_rate, decay_rate)                
        
        Z_, cache_ = multi_layer_forward(valX, parameters, drop_prob, "test")
        AL, cache_2_, val_cost = softmax_cross_entropy_loss(Z_, valY_one_hot)
    
        if ii % 10 == 0:
            costs.append(cost)
            val_costs.append(val_cost)
        if ii % 10 == 0:
            print("Cost at iteration %i is: %.05f, learning rate: %.05f" %(ii, cost, alpha))
    
    return costs, val_costs, parameters

In [ ]:
# Configuration 1 - Overfittting case, No dropout regularization

net_dims = [784,516,256]
net_dims.append(10) # Adding the digits layer with dimensionality = 10
print("Network dimensions are:" + str(net_dims))

# getting the subset dataset from MNIST
train_data, train_label, test_data, test_label, val_data, val_label = mnist(noTrSamples=train_samples,noValSamples= val_samples,noTsSamples=test_samples,digits= digits)

# initialize learning rate and num_iterations
learning_rate = .03
num_iterations = 500

drop_prob = 0
mode = 'train'

costs,val_costs, parameters = multi_layer_network(train_data, train_label,val_data, val_label, net_dims, drop_prob, mode, \
        num_iterations=num_iterations, learning_rate= learning_rate)

# compute the accuracy for training set and testing set


mode ='test'
train_Pred = classify(train_data, parameters,mode,drop_prob)
val_Pred = classify(val_data, parameters,mode,drop_prob)
test_Pred = classify(test_data, parameters,mode,drop_prob)
print(train_Pred.shape)


trAcc = ( 1 - np.count_nonzero(train_Pred - train_label ) / float(train_Pred.shape[1])) * 100 
valAcc = ( 1 - np.count_nonzero(val_Pred - val_label ) / float(val_Pred.shape[1])) * 100 
teAcc = ( 1 - np.count_nonzero(test_Pred - test_label ) / float(test_Pred.shape[1]) ) * 100
print("Accuracy for training set is {0:0.3f} %".format(trAcc))
print("Accuracy for validation set is {0:0.3f} %".format(valAcc))
print("Accuracy for testing set is {0:0.3f} %".format(teAcc))

X = range(0,num_iterations,10)
plt.plot(X,costs)
plt.plot(X,val_costs)
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.legend(['Training','Validation'])
plt.show()

In [ ]:
# Configuration 2 - using dropout regularization

net_dims = [784,516,256]
net_dims.append(10) # Adding the digits layer with dimensionality = 10
print("Network dimensions are:" + str(net_dims))

# getting the subset dataset from MNIST
train_data, train_label, test_data, test_label, val_data, val_label = mnist(noTrSamples=train_samples,noValSamples= val_samples,noTsSamples=test_samples,digits= digits)

# initialize learning rate and num_iterations
learning_rate = .03
num_iterations = 500

drop_prob = 0.2
mode = 'train'

costs,val_costs, parameters = multi_layer_network(train_data, train_label,val_data, val_label, net_dims, drop_prob, mode, \
        num_iterations=num_iterations, learning_rate= learning_rate)

# compute the accuracy for training set and testing set
mode ='test'
train_Pred = classify(train_data, parameters,mode,drop_prob)
val_Pred = classify(val_data, parameters,mode,drop_prob)
test_Pred = classify(test_data, parameters,mode,drop_prob)
print(train_Pred.shape)


trAcc = ( 1 - np.count_nonzero(train_Pred - train_label ) / float(train_Pred.shape[1])) * 100 
valAcc = ( 1 - np.count_nonzero(val_Pred - val_label ) / float(val_Pred.shape[1])) * 100 
teAcc = ( 1 - np.count_nonzero(test_Pred - test_label ) / float(test_Pred.shape[1]) ) * 100
print("Accuracy for training set is {0:0.3f} %".format(trAcc))
print("Accuracy for validation set is {0:0.3f} %".format(valAcc))
print("Accuracy for testing set is {0:0.3f} %".format(teAcc))

X = range(0,num_iterations,10)
plt.plot(X,costs)
plt.plot(X,val_costs)
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.legend(['Training','Validation'])
plt.show()